In [1]:
%pylab inline
import boto3
import pywren



config = pywren.wrenconfig.default()

logclient = boto3.client('logs', region_name=config['account']['aws_region'])

Populating the interactive namespace from numpy and matplotlib


In [2]:
wrenexec = pywren.default_executor()
def f(x):
    return x + 3
a = wrenexec.call_async(f, 10)

In [3]:
a.result()

13

In [9]:
wrenexec.get_logs(a)

[(1484782306587,
  u'START RequestId: 4700c02a-ddd6-11e6-a44d-1d8e6d45ba73 Version: $LATEST'),
 (1484782306597, u'invocation started'),
 (1484782307707,
  u'bucket= jonas-testbucket2 key= pywren.jobs/58de4dc2-3080-44a2-9bc1-b94332c79239/aggdata.pickle status:  5 bytes'),
 (1484782308778, u'func download complete'),
 (1484782308888, u'input data download complete'),
 (1484782308889,
  u'to_make= /tmp/pymodules/pywren/scripts m_path= pywren/scripts'),
 (1484782308889, u'creating /tmp/pymodules/pywren/scripts/pywrencli.py'),
 (1484782308889,
  u'to_make= /tmp/pymodules/pywren/cloudpickle m_path= pywren/cloudpickle'),
 (1484782308907,
  u'creating /tmp/pymodules/pywren/cloudpickle/module_dependency.py'),
 (1484782308907, u'to_make= /tmp/pymodules/glob2 m_path= glob2'),
 (1484782308907, u'creating /tmp/pymodules/glob2/impl.py'),
 (1484782308907, u'to_make= /tmp/pymodules/pywren m_path= pywren'),
 (1484782308907, u'creating /tmp/pymodules/pywren/__init__.py'),
 (1484782308907,
  u'to_make= /

In [6]:
log_group_name = a.run_status['log_group_name']
log_stream_name = a.run_status['log_stream_name']
lambda_function_name = a.invoke_status['lambda_function_name']
aws_request_id = a.run_status['aws_request_id']

log_events = logclient.get_log_events(
    logGroupName=log_group_name,
    logStreamName=log_stream_name,)


In [7]:
this_events_logs = []
in_this_event = False
for event in log_events['events']:
    start_string = "START RequestId: {}".format(aws_request_id)
    end_string = "REPORT RequestId: {}".format(aws_request_id)

    message = event['message'].strip()
    timestamp = int(event['timestamp'])
    if start_string in message:
        in_this_event = True
    elif end_string in message:
        in_this_event = False
        this_events_logs.append((timestamp, message))
        
    if in_this_event:
        this_events_logs.append((timestamp, message))
    
    #print "{} : {}".format(event['timestamp'], event['message'].strip())

In [8]:
for event in this_events_logs:
    print "{} : {}".format(*event)

1484782306587 : START RequestId: 4700c02a-ddd6-11e6-a44d-1d8e6d45ba73 Version: $LATEST
1484782306597 : invocation started
1484782307707 : bucket= jonas-testbucket2 key= pywren.jobs/58de4dc2-3080-44a2-9bc1-b94332c79239/aggdata.pickle status:  5 bytes
1484782308778 : func download complete
1484782308888 : input data download complete
1484782308889 : to_make= /tmp/pymodules/pywren/scripts m_path= pywren/scripts
1484782308889 : creating /tmp/pymodules/pywren/scripts/pywrencli.py
1484782308889 : to_make= /tmp/pymodules/pywren/cloudpickle m_path= pywren/cloudpickle
1484782308907 : creating /tmp/pymodules/pywren/cloudpickle/module_dependency.py
1484782308907 : to_make= /tmp/pymodules/glob2 m_path= glob2
1484782308907 : creating /tmp/pymodules/glob2/impl.py
1484782308907 : to_make= /tmp/pymodules/pywren m_path= pywren
1484782308907 : creating /tmp/pymodules/pywren/__init__.py
1484782308907 : to_make= /tmp/pymodules/pywren/cloudpickle m_path= pywren/cloudpickle
1484782308907 : creating /tmp/pym

In [35]:
a.run_status['aws_request_id']

u'4747d503-ddce-11e6-897e-872b858b572a'